#Summary-Bert for didgest and trend

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import AutoTokenizer, EncoderDecoderModel, T5ForConditionalGeneration

model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, do_basic_tokenize=False, strip_accents=False)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [3]:
import torch
from tqdm import tqdm


def preprocess(text):
    clean_text = []
    for char in text:
        if char.isalpha() or char == " ":
            clean_text.append(char)
        else:
            clean_text.append(" ")

    text = " ".join("".join(clean_text).split())
    return text


def tokenize(tokenizer, sentences: list, seq_len) -> tuple:
    input_ids = []
    attention_masks = []
    for row in tqdm(sentences):
        encoded_dict =  tokenizer.encode_plus(
            row,
            add_special_tokens=True,
            max_length=seq_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        input_ids.append(encoded_dict["input_ids"])

        attention_masks.append(encoded_dict["attention_mask"])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids, attention_masks



In [4]:
!pip uninstall xlrd -y
!pip install xlrd

Found existing installation: xlrd 2.0.1
Uninstalling xlrd-2.0.1:
  Successfully uninstalled xlrd-2.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


In [5]:
import pandas as pd

clerk = pd.read_csv("clerk_2.csv")
consultant = pd.read_csv("consultant_2.csv")
clerk

,title,full_text,link,date,trend
0,Предприниматели купили 500 тыс. лицензий на от...,Самыми востребованными оказались программные п...,https://www.klerk.ru/buh/news/537139/,2022-10-08,Интернет и IT
1,Скоро в России появятся мобильные грузоматы. И...,Грузоматы – это быстровозводимые цифровые конт...,https://www.klerk.ru/buh/news/537134/,2022-10-08,Экономика России
2,🚩 Получить аккредитацию IT-компания может с уч...,Один вариант предполагает наличие ПО в реестре...,https://www.klerk.ru/buh/news/537154/,2022-10-08,IT-компании
3,Работодатель не платит мобилизованному зарплат...,"Военнослужащему, призванному по мобилизации, п...",https://www.klerk.ru/buh/news/537159/,2022-10-08,Мобилизация
4,С 1 марта 2023 года перемещаться на электросам...,Правительство утвердило изменения в Правилах д...,https://www.klerk.ru/buh/news/537132/,2022-10-08,Общество
...,...,...,...,...,...
195,Уже аккредитованные IT-компании должны дать со...,Сделать это надо в течение месяца с момента вс...,https://www.klerk.ru/buh/news/536584/,2022-10-08,IT-компании
196,"Компаниям из ЕС, Британии, Норвегии и Украины ...",Постановление от 30.09.2022 № 1728 вступит в с...,https://www.klerk.ru/buh/news/536587/,2022-10-08,Политика
197,Форму для аккредитации IT-компаний создадут до...,Чтобы аккредитацию могли пройти не только круп...,https://www.klerk.ru/buh/news/536586/,2022-10-08,IT-компании
198,❗Утвердили новый порядок аккредитации IT-компаний,Новый порядок аккредитации IT-компаний утвержд...,https://www.klerk.ru/buh/news/536583/,2022-10-08,IT-компании


In [6]:
data = pd.concat([consultant, clerk]).dropna()
data

,title,full_text,link,date,trend
0,Обзор новых антикризисных мер за 1 – 7 октября,См. материалы о частичной мобилизации и новых ...,https://www.consultant.ru/legalnews/20528/,2022-10-8,Антикризисные меры - 2022
1,О приостановке и возобновлении трудового догов...,Сведения о приостановке и возобновлении трудов...,https://www.consultant.ru/legalnews/20541/,2022-10-7,"Частичная мобилизация, Изменения-2022"
2,В срок не подтвердили деятельность – суды отме...,Если страхователь вовремя не подтвердит деятел...,https://www.consultant.ru/legalnews/20505/,2022-10-7,Страховые взносы
3,Учреждения по-прежнему должны передавать Казна...,Казначейство временно не размещает на сайте ww...,https://www.consultant.ru/legalnews/20538/,2022-10-7,"Антикризисные меры - 2022, Автономные учрежден..."
4,Суд указал на неосновательное обогащение: в до...,Организации установили в договоре цену с учето...,https://www.consultant.ru/legalnews/20492/,2022-10-7,НДС
...,...,...,...,...,...
195,Уже аккредитованные IT-компании должны дать со...,Сделать это надо в течение месяца с момента вс...,https://www.klerk.ru/buh/news/536584/,2022-10-08,IT-компании
196,"Компаниям из ЕС, Британии, Норвегии и Украины ...",Постановление от 30.09.2022 № 1728 вступит в с...,https://www.klerk.ru/buh/news/536587/,2022-10-08,Политика
197,Форму для аккредитации IT-компаний создадут до...,Чтобы аккредитацию могли пройти не только круп...,https://www.klerk.ru/buh/news/536586/,2022-10-08,IT-компании
198,❗Утвердили новый порядок аккредитации IT-компаний,Новый порядок аккредитации IT-компаний утвержд...,https://www.klerk.ru/buh/news/536583/,2022-10-08,IT-компании


# Digests

In [7]:
texts = data["full_text"].apply(preprocess)
texts

0      См материалы о частичной мобилизации и новых а...
1      Сведения о приостановке и возобновлении трудов...
2      Если страхователь вовремя не подтвердит деятел...
3      Казначейство временно не размещает на сайте ww...
4      Организации установили в договоре цену с учето...
                             ...                        
195    Сделать это надо в течение месяца с момента вс...
196    Постановление от вступит в силу октября Под за...
197    Чтобы аккредитацию могли пройти не только круп...
198    Новый порядок аккредитации IT компаний утвержд...
199    Студентов коммерческих вузов имеющих государст...
Name: full_text, Length: 397, dtype: object

In [11]:
input_ids, attention_masks = tokenize(tokenizer, list(texts), 128)

100%|██████████| 397/397 [00:00<00:00, 924.42it/s]


In [12]:
torch.cuda.empty_cache()

In [13]:
!nvidia-smi

Sun Oct  9 02:07:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
device = torch.device("cuda")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(30000, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(30000, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [15]:
digests = []
for i in tqdm(range(397)):
    output_ids = model.generate(
        input_ids=torch.unsqueeze(input_ids[i], 0).to(device),
        attention_mask=torch.unsqueeze(attention_masks[i], 0).to(device),
        max_length=64,
        no_repeat_ngram_size=3,
        num_beams=3,
        top_p=0.9
    )[0].detach().cpu().numpy()
    
    digest = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    digests.append(digest)

digests

100%|██████████| 397/397 [08:16<00:00,  1.25s/it]


['С октября и до конца года запретили перемещать грузы по территории РФ автотранспортом иностранных перевозчиков. От плановых проверок освободят предприятия и организации деятельность которых не относится к категориям чрезвычайно высокого и высокого',
 'ПФР предложил изменить форму СЗВ ТД для граждан, заключивших с февраля по сентября контракт о военной службе или добровольном содействии ВС РФ и уволенных в этот период. Таким образом, граждане заключившие с ',
 'ФСС устанавливает ему тариф взносов по деятельности из ЕГРЮЛ с самым высоким классом риска Если страхователь вовремя не подтвердит деятельность ФСС, фонд доначисляет ему взносы исходя из деятельности которую страхователь реально в',
 'Казначейство временно не размещает на сайте www bus gov ru планы ФХД и отчеты о результатах деятельности и об использовании имущества ведомства Документы будут хранить в закрытом доступе и использовать в т ч при',
 'Продавцы установили в договоре цену с учетом НДС После оплаты продавец передал пок

In [16]:
data["digest"] = digests
data

,title,full_text,link,date,trend,digest
0,Обзор новых антикризисных мер за 1 – 7 октября,См. материалы о частичной мобилизации и новых ...,https://www.consultant.ru/legalnews/20528/,2022-10-8,Антикризисные меры - 2022,С октября и до конца года запретили перемещать...
1,О приостановке и возобновлении трудового догов...,Сведения о приостановке и возобновлении трудов...,https://www.consultant.ru/legalnews/20541/,2022-10-7,"Частичная мобилизация, Изменения-2022",ПФР предложил изменить форму СЗВ ТД для гражда...
2,В срок не подтвердили деятельность – суды отме...,Если страхователь вовремя не подтвердит деятел...,https://www.consultant.ru/legalnews/20505/,2022-10-7,Страховые взносы,ФСС устанавливает ему тариф взносов по деятель...
3,Учреждения по-прежнему должны передавать Казна...,Казначейство временно не размещает на сайте ww...,https://www.consultant.ru/legalnews/20538/,2022-10-7,"Антикризисные меры - 2022, Автономные учрежден...",Казначейство временно не размещает на сайте ww...
4,Суд указал на неосновательное обогащение: в до...,Организации установили в договоре цену с учето...,https://www.consultant.ru/legalnews/20492/,2022-10-7,НДС,Продавцы установили в договоре цену с учетом Н...
...,...,...,...,...,...,...
195,Уже аккредитованные IT-компании должны дать со...,Сделать это надо в течение месяца с момента вс...,https://www.klerk.ru/buh/news/536584/,2022-10-08,IT-компании,В течение месяца с момента вступления в силу н...
196,"Компаниям из ЕС, Британии, Норвегии и Украины ...",Постановление от 30.09.2022 № 1728 вступит в с...,https://www.klerk.ru/buh/news/536587/,2022-10-08,Политика,Вступит в силу запрет на перевозки грузоперево...
197,Форму для аккредитации IT-компаний создадут до...,Чтобы аккредитацию могли пройти не только круп...,https://www.klerk.ru/buh/news/536586/,2022-10-08,IT-компании,Минцифры разработали два варианта аккредитации...
198,❗Утвердили новый порядок аккредитации IT-компаний,Новый порядок аккредитации IT-компаний утвержд...,https://www.klerk.ru/buh/news/536583/,2022-10-08,IT-компании,Новый порядок аккредитации IT компаний утвержд...


# Embs

In [18]:
from transformers import AutoTokenizer, RobertaTokenizerFast, EncoderDecoderModel, T5ForConditionalGeneration, AutoModel

model_name = "sberbank-ai/ruRoberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at sberbank-ai/ruRoberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at sberbank-ai/ruRoberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to b

In [20]:
input_ids, attention_masks = tokenize(tokenizer, list(texts), 64)

100%|██████████| 397/397 [00:00<00:00, 950.43it/s] 


In [21]:
import numpy as np


embeddings = []
model.eval()
with torch.no_grad():
    for i in tqdm(range(len(input_ids))):
        output = model(
          input_ids=torch.unsqueeze(input_ids[i], 0), 
          attention_mask=torch.unsqueeze(attention_masks[i], 0)
        )["pooler_output"].detach().cpu().numpy()
        
        embeddings.append(output)

embeddings = np.vstack(embeddings)

100%|██████████| 397/397 [03:02<00:00,  2.18it/s]


In [22]:
!pip list

Package                       Version
----------------------------- ----------------------------
absl-py                       1.2.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.10.3
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.8

In [23]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=32, algorithm="arpack")
svd = svd.fit(embeddings)
svd_embeddings = svd.transform(embeddings)

In [24]:
import joblib
joblib.dump(svd, "svd.joblib")

['svd.joblib']

In [25]:
embeddings_df = pd.DataFrame(svd_embeddings, columns=[f"feature_{i}" for i in range(32)])
embeddings_df                            

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,13.849458,0.732103,0.476175,0.755176,0.757966,-1.721852,1.847961,-1.125121,1.048933,-1.289282,...,1.337525,0.577069,-0.602152,-0.435522,0.574146,-1.130774,-0.090963,0.899269,-1.132337,-0.289081
1,13.607505,-3.370009,1.459685,1.630351,0.513634,0.585442,0.182015,2.610637,0.397811,-0.990277,...,0.207408,0.166558,-0.810546,-0.605356,0.445859,-0.511923,-0.052084,-0.694921,0.127944,-0.842587
2,13.750451,-2.842947,-1.469127,-1.065693,-0.282129,0.148672,0.688446,-0.934232,0.218967,0.293445,...,-0.568682,0.163121,-0.522541,-0.995014,0.486491,0.758738,0.311415,0.722057,0.625320,-0.434852
3,14.384837,0.639996,-0.591019,0.107613,-0.900631,0.275584,0.060898,-0.272893,-0.848182,-1.206136,...,0.088567,0.226906,0.062146,-0.205083,0.443507,0.463632,-0.050001,-0.259821,-0.728171,-0.488235
4,13.588634,-3.312975,-1.322595,-2.108049,-1.184054,-0.307121,-0.256404,-0.597792,0.568406,0.847311,...,-0.686407,0.285492,0.265495,-0.638079,0.233661,-0.024538,0.311356,0.157094,-0.565702,0.616543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,14.090535,0.465328,0.564796,1.845265,-1.429395,0.157692,0.332082,1.895400,-2.184567,-0.206815,...,0.301745,-0.000264,0.625704,-1.134227,-0.126880,-0.160556,0.243631,-0.810310,-0.306426,0.308330
393,13.677405,2.016342,0.358703,0.720636,-1.434172,-1.881539,1.318663,1.216004,0.700777,0.979178,...,-0.258205,-0.598731,-1.375967,-1.055371,-0.677902,0.512231,-0.525778,-0.475386,0.391788,0.103907
394,14.335608,1.770023,0.395624,0.212348,-2.053525,0.993419,-0.865665,0.477995,-1.650551,0.572590,...,-0.395892,-0.649326,0.265403,0.060237,-0.145965,-0.373918,0.409181,1.145438,0.084850,-0.026938
395,14.167031,0.135683,0.963204,0.831027,-1.703341,0.104466,-0.108340,-0.165882,-0.476582,0.816192,...,-0.659683,-0.550436,0.656242,-0.245635,-0.754002,-0.373111,-0.157214,0.438322,-0.044224,0.511780


In [26]:
data

,title,full_text,link,date,trend,digest
0,Обзор новых антикризисных мер за 1 – 7 октября,См. материалы о частичной мобилизации и новых ...,https://www.consultant.ru/legalnews/20528/,2022-10-8,Антикризисные меры - 2022,С октября и до конца года запретили перемещать...
1,О приостановке и возобновлении трудового догов...,Сведения о приостановке и возобновлении трудов...,https://www.consultant.ru/legalnews/20541/,2022-10-7,"Частичная мобилизация, Изменения-2022",ПФР предложил изменить форму СЗВ ТД для гражда...
2,В срок не подтвердили деятельность – суды отме...,Если страхователь вовремя не подтвердит деятел...,https://www.consultant.ru/legalnews/20505/,2022-10-7,Страховые взносы,ФСС устанавливает ему тариф взносов по деятель...
3,Учреждения по-прежнему должны передавать Казна...,Казначейство временно не размещает на сайте ww...,https://www.consultant.ru/legalnews/20538/,2022-10-7,"Антикризисные меры - 2022, Автономные учрежден...",Казначейство временно не размещает на сайте ww...
4,Суд указал на неосновательное обогащение: в до...,Организации установили в договоре цену с учето...,https://www.consultant.ru/legalnews/20492/,2022-10-7,НДС,Продавцы установили в договоре цену с учетом Н...
...,...,...,...,...,...,...
195,Уже аккредитованные IT-компании должны дать со...,Сделать это надо в течение месяца с момента вс...,https://www.klerk.ru/buh/news/536584/,2022-10-08,IT-компании,В течение месяца с момента вступления в силу н...
196,"Компаниям из ЕС, Британии, Норвегии и Украины ...",Постановление от 30.09.2022 № 1728 вступит в с...,https://www.klerk.ru/buh/news/536587/,2022-10-08,Политика,Вступит в силу запрет на перевозки грузоперево...
197,Форму для аккредитации IT-компаний создадут до...,Чтобы аккредитацию могли пройти не только круп...,https://www.klerk.ru/buh/news/536586/,2022-10-08,IT-компании,Минцифры разработали два варианта аккредитации...
198,❗Утвердили новый порядок аккредитации IT-компаний,Новый порядок аккредитации IT-компаний утвержд...,https://www.klerk.ru/buh/news/536583/,2022-10-08,IT-компании,Новый порядок аккредитации IT компаний утвержд...


In [27]:
news_bd = pd.concat([data.reset_index(drop=True), embeddings_df.reset_index(drop=True)],  axis=1)
news_bd

,title,full_text,link,date,trend,digest,feature_0,feature_1,feature_2,feature_3,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,Обзор новых антикризисных мер за 1 – 7 октября,См. материалы о частичной мобилизации и новых ...,https://www.consultant.ru/legalnews/20528/,2022-10-8,Антикризисные меры - 2022,С октября и до конца года запретили перемещать...,13.849458,0.732103,0.476175,0.755176,...,1.337525,0.577069,-0.602152,-0.435522,0.574146,-1.130774,-0.090963,0.899269,-1.132337,-0.289081
1,О приостановке и возобновлении трудового догов...,Сведения о приостановке и возобновлении трудов...,https://www.consultant.ru/legalnews/20541/,2022-10-7,"Частичная мобилизация, Изменения-2022",ПФР предложил изменить форму СЗВ ТД для гражда...,13.607505,-3.370009,1.459685,1.630351,...,0.207408,0.166558,-0.810546,-0.605356,0.445859,-0.511923,-0.052084,-0.694921,0.127944,-0.842587
2,В срок не подтвердили деятельность – суды отме...,Если страхователь вовремя не подтвердит деятел...,https://www.consultant.ru/legalnews/20505/,2022-10-7,Страховые взносы,ФСС устанавливает ему тариф взносов по деятель...,13.750451,-2.842947,-1.469127,-1.065693,...,-0.568682,0.163121,-0.522541,-0.995014,0.486491,0.758738,0.311415,0.722057,0.625320,-0.434852
3,Учреждения по-прежнему должны передавать Казна...,Казначейство временно не размещает на сайте ww...,https://www.consultant.ru/legalnews/20538/,2022-10-7,"Антикризисные меры - 2022, Автономные учрежден...",Казначейство временно не размещает на сайте ww...,14.384837,0.639996,-0.591019,0.107613,...,0.088567,0.226906,0.062146,-0.205083,0.443507,0.463632,-0.050001,-0.259821,-0.728171,-0.488235
4,Суд указал на неосновательное обогащение: в до...,Организации установили в договоре цену с учето...,https://www.consultant.ru/legalnews/20492/,2022-10-7,НДС,Продавцы установили в договоре цену с учетом Н...,13.588634,-3.312975,-1.322595,-2.108049,...,-0.686407,0.285492,0.265495,-0.638079,0.233661,-0.024538,0.311356,0.157094,-0.565702,0.616543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Уже аккредитованные IT-компании должны дать со...,Сделать это надо в течение месяца с момента вс...,https://www.klerk.ru/buh/news/536584/,2022-10-08,IT-компании,В течение месяца с момента вступления в силу н...,14.090535,0.465328,0.564796,1.845265,...,0.301745,-0.000264,0.625704,-1.134227,-0.126880,-0.160556,0.243631,-0.810310,-0.306426,0.308330
393,"Компаниям из ЕС, Британии, Норвегии и Украины ...",Постановление от 30.09.2022 № 1728 вступит в с...,https://www.klerk.ru/buh/news/536587/,2022-10-08,Политика,Вступит в силу запрет на перевозки грузоперево...,13.677405,2.016342,0.358703,0.720636,...,-0.258205,-0.598731,-1.375967,-1.055371,-0.677902,0.512231,-0.525778,-0.475386,0.391788,0.103907
394,Форму для аккредитации IT-компаний создадут до...,Чтобы аккредитацию могли пройти не только круп...,https://www.klerk.ru/buh/news/536586/,2022-10-08,IT-компании,Минцифры разработали два варианта аккредитации...,14.335608,1.770023,0.395624,0.212348,...,-0.395892,-0.649326,0.265403,0.060237,-0.145965,-0.373918,0.409181,1.145438,0.084850,-0.026938
395,❗Утвердили новый порядок аккредитации IT-компаний,Новый порядок аккредитации IT-компаний утвержд...,https://www.klerk.ru/buh/news/536583/,2022-10-08,IT-компании,Новый порядок аккредитации IT компаний утвержд...,14.167031,0.135683,0.963204,0.831027,...,-0.659683,-0.550436,0.656242,-0.245635,-0.754002,-0.373111,-0.157214,0.438322,-0.044224,0.511780


# CatBoost

In [ ]:
from catboost import CatBoostClassifier
cbc_buh = CatBoostClassifier()
cbc_buh.fit(news_bd[[f"feature_{i}" for i in range(32)]], news_bd["buh_insight"])

In [253]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict_proba(X)[:, 1])

cross_val_score(CatBoostClassifier(verbose=0), X=news_bd[[f"feature_{i}" for i in range(32)]], y=news_bd["business_insight"], cv=5, scoring=scorer)

array([0.54573935, 0.45927318, 0.56453634, 0.59710874, 0.54890605])

In [254]:
cbc_business = CatBoostClassifier()
cbc_business.fit(news_bd[[f"feature_{i}" for i in range(32)]], news_bd["business_insight"])

Learning rate set to 0.006959
0:	learn: 0.6917328	total: 4.34ms	remaining: 4.33s
1:	learn: 0.6903420	total: 8.85ms	remaining: 4.42s
2:	learn: 0.6889275	total: 13ms	remaining: 4.31s
3:	learn: 0.6872202	total: 17.2ms	remaining: 4.27s
4:	learn: 0.6858598	total: 21.2ms	remaining: 4.23s
5:	learn: 0.6848249	total: 25.3ms	remaining: 4.19s
6:	learn: 0.6841593	total: 29.5ms	remaining: 4.18s
7:	learn: 0.6826755	total: 33.6ms	remaining: 4.17s
8:	learn: 0.6817418	total: 37.9ms	remaining: 4.17s
9:	learn: 0.6805122	total: 42.1ms	remaining: 4.16s
10:	learn: 0.6793558	total: 46.6ms	remaining: 4.19s
11:	learn: 0.6781330	total: 50.6ms	remaining: 4.16s
12:	learn: 0.6769875	total: 54.6ms	remaining: 4.15s
13:	learn: 0.6760058	total: 58.8ms	remaining: 4.14s
14:	learn: 0.6751984	total: 62.7ms	remaining: 4.12s
15:	learn: 0.6736855	total: 66.9ms	remaining: 4.12s
16:	learn: 0.6723779	total: 71.1ms	remaining: 4.11s
17:	learn: 0.6713023	total: 75.3ms	remaining: 4.11s
18:	learn: 0.6700152	total: 79.5ms	remaining: 

In [265]:
from datetime import datetime
cbc_buh.save_model(f"{datetime.now()}.cbm")
cbc_business.save_model(f"{datetime.now()}.cbm")

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 30 kB/s 


In [289]:
news_bd["buh_likes"] = [0 for i in range(397)]
news_bd["business_likes"] = [0 for i in range(397)]

In [28]:
def change_date(date):
    y, m, d = date = date.split("-")
    if len(m) < 2:
        m = f"0{m}"
    if len(d) < 2:
        d = f"0{d}"
    
    return "-".join([y, m, d])


In [29]:
news_bd["date"] = news_bd["date"].apply(change_date)
news_bd

,title,full_text,link,date,trend,digest,feature_0,feature_1,feature_2,feature_3,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,Обзор новых антикризисных мер за 1 – 7 октября,См. материалы о частичной мобилизации и новых ...,https://www.consultant.ru/legalnews/20528/,2022-10-08,Антикризисные меры - 2022,С октября и до конца года запретили перемещать...,13.849458,0.732103,0.476175,0.755176,...,1.337525,0.577069,-0.602152,-0.435522,0.574146,-1.130774,-0.090963,0.899269,-1.132337,-0.289081
1,О приостановке и возобновлении трудового догов...,Сведения о приостановке и возобновлении трудов...,https://www.consultant.ru/legalnews/20541/,2022-10-07,"Частичная мобилизация, Изменения-2022",ПФР предложил изменить форму СЗВ ТД для гражда...,13.607505,-3.370009,1.459685,1.630351,...,0.207408,0.166558,-0.810546,-0.605356,0.445859,-0.511923,-0.052084,-0.694921,0.127944,-0.842587
2,В срок не подтвердили деятельность – суды отме...,Если страхователь вовремя не подтвердит деятел...,https://www.consultant.ru/legalnews/20505/,2022-10-07,Страховые взносы,ФСС устанавливает ему тариф взносов по деятель...,13.750451,-2.842947,-1.469127,-1.065693,...,-0.568682,0.163121,-0.522541,-0.995014,0.486491,0.758738,0.311415,0.722057,0.625320,-0.434852
3,Учреждения по-прежнему должны передавать Казна...,Казначейство временно не размещает на сайте ww...,https://www.consultant.ru/legalnews/20538/,2022-10-07,"Антикризисные меры - 2022, Автономные учрежден...",Казначейство временно не размещает на сайте ww...,14.384837,0.639996,-0.591019,0.107613,...,0.088567,0.226906,0.062146,-0.205083,0.443507,0.463632,-0.050001,-0.259821,-0.728171,-0.488235
4,Суд указал на неосновательное обогащение: в до...,Организации установили в договоре цену с учето...,https://www.consultant.ru/legalnews/20492/,2022-10-07,НДС,Продавцы установили в договоре цену с учетом Н...,13.588634,-3.312975,-1.322595,-2.108049,...,-0.686407,0.285492,0.265495,-0.638079,0.233661,-0.024538,0.311356,0.157094,-0.565702,0.616543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Уже аккредитованные IT-компании должны дать со...,Сделать это надо в течение месяца с момента вс...,https://www.klerk.ru/buh/news/536584/,2022-10-08,IT-компании,В течение месяца с момента вступления в силу н...,14.090535,0.465328,0.564796,1.845265,...,0.301745,-0.000264,0.625704,-1.134227,-0.126880,-0.160556,0.243631,-0.810310,-0.306426,0.308330
393,"Компаниям из ЕС, Британии, Норвегии и Украины ...",Постановление от 30.09.2022 № 1728 вступит в с...,https://www.klerk.ru/buh/news/536587/,2022-10-08,Политика,Вступит в силу запрет на перевозки грузоперево...,13.677405,2.016342,0.358703,0.720636,...,-0.258205,-0.598731,-1.375967,-1.055371,-0.677902,0.512231,-0.525778,-0.475386,0.391788,0.103907
394,Форму для аккредитации IT-компаний создадут до...,Чтобы аккредитацию могли пройти не только круп...,https://www.klerk.ru/buh/news/536586/,2022-10-08,IT-компании,Минцифры разработали два варианта аккредитации...,14.335608,1.770023,0.395624,0.212348,...,-0.395892,-0.649326,0.265403,0.060237,-0.145965,-0.373918,0.409181,1.145438,0.084850,-0.026938
395,❗Утвердили новый порядок аккредитации IT-компаний,Новый порядок аккредитации IT-компаний утвержд...,https://www.klerk.ru/buh/news/536583/,2022-10-08,IT-компании,Новый порядок аккредитации IT компаний утвержд...,14.167031,0.135683,0.963204,0.831027,...,-0.659683,-0.550436,0.656242,-0.245635,-0.754002,-0.373111,-0.157214,0.438322,-0.044224,0.511780


In [260]:
news_bd = news_bd.drop(labels=["buh_insight", "business_insight"], axis=1)
news_bd

,title,full_text,link,date,digest,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,buh_likes,business_likes
0,Разъяснили особенности учета занятости доброво...,"Росстат указал, что мобилизованных и доброволь...",https://www.consultant.ru/legalnews/20519/,2022-10-06,Росстат указал что мобилизованных и добровольн...,7.984824,12.751231,-0.275211,-0.016271,-0.878433,...,0.060125,-1.873767,-0.143583,-0.708169,0.018905,-0.160037,0.348027,0.582351,0,0
1,В окончательном чтении приняли поправки об ипо...,Предлагают размещать деньги в составе ипотечно...,https://www.consultant.ru/legalnews/20515/,2022-10-06,Эксперты предлагают размещать деньги в составе...,13.037721,-1.953721,0.284633,4.668533,-0.029261,...,0.812001,-1.066263,-0.466455,0.471710,-0.443539,0.127607,-0.392884,-0.569936,0,0
2,Стали известны возможные коэффициенты-дефлятор...,Минэкономразвития выставило на общественное об...,https://www.consultant.ru/legalnews/20507/,2022-10-06,Минэкономразвития выставило на общественное об...,13.247151,-3.513483,2.630593,-0.639810,2.996490,...,-0.191367,0.085127,0.644895,0.427077,0.999138,0.072361,-0.595821,-0.690278,0,0
3,Подготовили поправки к перечням КБК на 2022 го...,Проект проходит общественное обсуждение до 17 ...,https://www.consultant.ru/legalnews/20512/,2022-10-06,В списки кодов доходов бюджетов и целевых стат...,13.589590,-2.658904,1.394210,-0.252348,1.948244,...,-0.459939,0.390438,-0.819117,0.228050,-0.020272,0.337856,-0.535990,0.476796,0,0
4,Планируют смягчить требования к энергоэффектив...,Предложили при приобретении телевизоров с кодо...,https://www.consultant.ru/legalnews/20513/,2022-10-06,Минэкономразвития предлагает при приобретении ...,13.425740,-2.203631,2.189778,-1.291170,0.020358,...,1.076322,0.003372,0.135697,-0.510732,-0.411073,-0.787649,-0.147748,-1.199033,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,"Сотрудники, которых мобилизовали в отпуске, до...","По мнению ведомства, военная служба по мобилиз...",https://www.klerk.ru/buh/news/536573/,2022-10-07,Военная служба по мобилизации является государ...,13.436013,-2.769251,-3.340399,-1.554684,1.454680,...,0.584011,0.088175,1.022828,-0.507575,0.205682,0.012518,-0.067091,-0.047683,0,0
395,Мобилизованных ИП предлагают освободить от фик...,Депутаты от ЛДПР внесли в Госдуму законопроект...,https://www.klerk.ru/buh/news/536571/,2022-10-07,Депутаты от ЛДПР внесли в Госдуму законопроект...,13.787848,-2.632453,-0.606050,-0.016684,1.398054,...,0.211130,-0.605277,1.003982,-1.138405,-0.874625,-0.347971,0.620838,-0.470176,0,0
396,Lego сменит название в России. Теперь магазины...,На месте сертифицированных магазинов Lego тепе...,https://www.klerk.ru/buh/news/536569/,2022-10-07,На месте сертифицированных магазинов Lego тепе...,12.551700,-2.650238,2.800444,0.272409,-3.066269,...,-1.149882,-0.395721,0.626006,1.041503,-0.350941,-0.592994,0.900423,0.310449,0,0
397,"Телефонные мошенники предлагают подработку, а ...",Мошенники звонят по телефону и предлагают удал...,https://www.klerk.ru/buh/news/536558/,2022-10-07,Мошенники звонят по телефону и предлагают удал...,12.662375,-2.722646,1.211371,-0.640612,-2.724880,...,-0.009787,0.185656,-0.405690,-0.079283,0.320376,0.547683,-0.266966,0.325100,0,0


In [30]:
news_bd["news_id"] = [i for i in range(397)]
news_bd

,title,full_text,link,date,trend,digest,feature_0,feature_1,feature_2,feature_3,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,news_id
0,Обзор новых антикризисных мер за 1 – 7 октября,См. материалы о частичной мобилизации и новых ...,https://www.consultant.ru/legalnews/20528/,2022-10-08,Антикризисные меры - 2022,С октября и до конца года запретили перемещать...,13.849458,0.732103,0.476175,0.755176,...,0.577069,-0.602152,-0.435522,0.574146,-1.130774,-0.090963,0.899269,-1.132337,-0.289081,0
1,О приостановке и возобновлении трудового догов...,Сведения о приостановке и возобновлении трудов...,https://www.consultant.ru/legalnews/20541/,2022-10-07,"Частичная мобилизация, Изменения-2022",ПФР предложил изменить форму СЗВ ТД для гражда...,13.607505,-3.370009,1.459685,1.630351,...,0.166558,-0.810546,-0.605356,0.445859,-0.511923,-0.052084,-0.694921,0.127944,-0.842587,1
2,В срок не подтвердили деятельность – суды отме...,Если страхователь вовремя не подтвердит деятел...,https://www.consultant.ru/legalnews/20505/,2022-10-07,Страховые взносы,ФСС устанавливает ему тариф взносов по деятель...,13.750451,-2.842947,-1.469127,-1.065693,...,0.163121,-0.522541,-0.995014,0.486491,0.758738,0.311415,0.722057,0.625320,-0.434852,2
3,Учреждения по-прежнему должны передавать Казна...,Казначейство временно не размещает на сайте ww...,https://www.consultant.ru/legalnews/20538/,2022-10-07,"Антикризисные меры - 2022, Автономные учрежден...",Казначейство временно не размещает на сайте ww...,14.384837,0.639996,-0.591019,0.107613,...,0.226906,0.062146,-0.205083,0.443507,0.463632,-0.050001,-0.259821,-0.728171,-0.488235,3
4,Суд указал на неосновательное обогащение: в до...,Организации установили в договоре цену с учето...,https://www.consultant.ru/legalnews/20492/,2022-10-07,НДС,Продавцы установили в договоре цену с учетом Н...,13.588634,-3.312975,-1.322595,-2.108049,...,0.285492,0.265495,-0.638079,0.233661,-0.024538,0.311356,0.157094,-0.565702,0.616543,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Уже аккредитованные IT-компании должны дать со...,Сделать это надо в течение месяца с момента вс...,https://www.klerk.ru/buh/news/536584/,2022-10-08,IT-компании,В течение месяца с момента вступления в силу н...,14.090535,0.465328,0.564796,1.845265,...,-0.000264,0.625704,-1.134227,-0.126880,-0.160556,0.243631,-0.810310,-0.306426,0.308330,392
393,"Компаниям из ЕС, Британии, Норвегии и Украины ...",Постановление от 30.09.2022 № 1728 вступит в с...,https://www.klerk.ru/buh/news/536587/,2022-10-08,Политика,Вступит в силу запрет на перевозки грузоперево...,13.677405,2.016342,0.358703,0.720636,...,-0.598731,-1.375967,-1.055371,-0.677902,0.512231,-0.525778,-0.475386,0.391788,0.103907,393
394,Форму для аккредитации IT-компаний создадут до...,Чтобы аккредитацию могли пройти не только круп...,https://www.klerk.ru/buh/news/536586/,2022-10-08,IT-компании,Минцифры разработали два варианта аккредитации...,14.335608,1.770023,0.395624,0.212348,...,-0.649326,0.265403,0.060237,-0.145965,-0.373918,0.409181,1.145438,0.084850,-0.026938,394
395,❗Утвердили новый порядок аккредитации IT-компаний,Новый порядок аккредитации IT-компаний утвержд...,https://www.klerk.ru/buh/news/536583/,2022-10-08,IT-компании,Новый порядок аккредитации IT компаний утвержд...,14.167031,0.135683,0.963204,0.831027,...,-0.550436,0.656242,-0.245635,-0.754002,-0.373111,-0.157214,0.438322,-0.044224,0.511780,395


In [31]:
cols = news_bd.columns.tolist()
cols = cols[-1:] + cols[:-1]
news_bd[cols]

,news_id,title,full_text,link,date,trend,digest,feature_0,feature_1,feature_2,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,0,Обзор новых антикризисных мер за 1 – 7 октября,См. материалы о частичной мобилизации и новых ...,https://www.consultant.ru/legalnews/20528/,2022-10-08,Антикризисные меры - 2022,С октября и до конца года запретили перемещать...,13.849458,0.732103,0.476175,...,1.337525,0.577069,-0.602152,-0.435522,0.574146,-1.130774,-0.090963,0.899269,-1.132337,-0.289081
1,1,О приостановке и возобновлении трудового догов...,Сведения о приостановке и возобновлении трудов...,https://www.consultant.ru/legalnews/20541/,2022-10-07,"Частичная мобилизация, Изменения-2022",ПФР предложил изменить форму СЗВ ТД для гражда...,13.607505,-3.370009,1.459685,...,0.207408,0.166558,-0.810546,-0.605356,0.445859,-0.511923,-0.052084,-0.694921,0.127944,-0.842587
2,2,В срок не подтвердили деятельность – суды отме...,Если страхователь вовремя не подтвердит деятел...,https://www.consultant.ru/legalnews/20505/,2022-10-07,Страховые взносы,ФСС устанавливает ему тариф взносов по деятель...,13.750451,-2.842947,-1.469127,...,-0.568682,0.163121,-0.522541,-0.995014,0.486491,0.758738,0.311415,0.722057,0.625320,-0.434852
3,3,Учреждения по-прежнему должны передавать Казна...,Казначейство временно не размещает на сайте ww...,https://www.consultant.ru/legalnews/20538/,2022-10-07,"Антикризисные меры - 2022, Автономные учрежден...",Казначейство временно не размещает на сайте ww...,14.384837,0.639996,-0.591019,...,0.088567,0.226906,0.062146,-0.205083,0.443507,0.463632,-0.050001,-0.259821,-0.728171,-0.488235
4,4,Суд указал на неосновательное обогащение: в до...,Организации установили в договоре цену с учето...,https://www.consultant.ru/legalnews/20492/,2022-10-07,НДС,Продавцы установили в договоре цену с учетом Н...,13.588634,-3.312975,-1.322595,...,-0.686407,0.285492,0.265495,-0.638079,0.233661,-0.024538,0.311356,0.157094,-0.565702,0.616543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,392,Уже аккредитованные IT-компании должны дать со...,Сделать это надо в течение месяца с момента вс...,https://www.klerk.ru/buh/news/536584/,2022-10-08,IT-компании,В течение месяца с момента вступления в силу н...,14.090535,0.465328,0.564796,...,0.301745,-0.000264,0.625704,-1.134227,-0.126880,-0.160556,0.243631,-0.810310,-0.306426,0.308330
393,393,"Компаниям из ЕС, Британии, Норвегии и Украины ...",Постановление от 30.09.2022 № 1728 вступит в с...,https://www.klerk.ru/buh/news/536587/,2022-10-08,Политика,Вступит в силу запрет на перевозки грузоперево...,13.677405,2.016342,0.358703,...,-0.258205,-0.598731,-1.375967,-1.055371,-0.677902,0.512231,-0.525778,-0.475386,0.391788,0.103907
394,394,Форму для аккредитации IT-компаний создадут до...,Чтобы аккредитацию могли пройти не только круп...,https://www.klerk.ru/buh/news/536586/,2022-10-08,IT-компании,Минцифры разработали два варианта аккредитации...,14.335608,1.770023,0.395624,...,-0.395892,-0.649326,0.265403,0.060237,-0.145965,-0.373918,0.409181,1.145438,0.084850,-0.026938
395,395,❗Утвердили новый порядок аккредитации IT-компаний,Новый порядок аккредитации IT-компаний утвержд...,https://www.klerk.ru/buh/news/536583/,2022-10-08,IT-компании,Новый порядок аккредитации IT компаний утвержд...,14.167031,0.135683,0.963204,...,-0.659683,-0.550436,0.656242,-0.245635,-0.754002,-0.373111,-0.157214,0.438322,-0.044224,0.511780


In [296]:
news_bd[cols].to_excel("news_2.xls", index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  """Entry point for launching an IPython kernel.


# Insights

In [32]:
X = news_bd[[f"feature_{i}" for i in range(32)]].to_numpy()

In [33]:
from sklearn.neighbors import KDTree
from scipy.spatial.distance import cosine

tree = KDTree(X, leaf_size=2)

In [34]:
closest = tree.query(X, k=2, return_distance=False)[:, 1]
closest

array([ 59,  98, 169, 366, 112, 160, 178,  16, 106,  93, 201,  98,  75,
       231, 126, 184,   7, 111, 100, 126,  90, 266,  12, 311, 327, 133,
       166, 303, 144, 163, 153, 178, 192, 350, 323,  37,  82, 332,  52,
       291, 113, 293, 345,  53, 327, 168,  81, 267, 121, 171, 122, 117,
        38, 111, 300, 206,  67, 231,  83, 380,  62, 192,  60, 130,  99,
        77, 291, 185, 138, 184,  45,  63,  62, 384, 149,  12, 300, 379,
       291, 223,  83,  85,  36,  87, 167,  81, 185, 130,  77, 149, 396,
       187, 347,   9, 346, 132,  81,  26,  11, 118, 103, 310, 151,  31,
       152, 371,   8, 247,   8, 227,  67, 162,  40,  40, 337, 105, 124,
        51,  99, 109, 396,  48,  50, 130, 155,  82,  14, 366,  65, 172,
        63, 239,  95, 323,  63, 320, 195, 320, 179, 279,  99,  26, 130,
        44,  28,   8, 299, 243, 188, 291,  31, 102, 104, 336, 178, 124,
       178, 320,  16, 123,   5, 396, 111,  29,  30,  31,  26, 391,  45,
       163, 117, 130, 129,  72, 191,  38,   9, 347,  31, 138, 16

In [35]:
f_digests = list(news_bd["digest"])
t_digests = list(news_bd["digest"].loc[closest])
f_dates = list(news_bd["date"])
t_dates = list(news_bd["date"].loc[closest])

In [36]:
pairs = []
for i in range(len(f_dates)):
    if f_dates[i] < t_dates[i]:
        pairs.append([f_digests[i], t_digests[i]])
    else:
        pairs.append([t_digests[i], f_digests[i]])

pairs

[['Правительство не будет проводить плановые проверки предприятий и организаций деятельность которых не относится к категориям чрезвычайно высокого и высокого риска и не являются опасными производственными объектами II класса опасности и гидротехническими с',
  'С октября и до конца года запретили перемещать грузы по территории РФ автотранспортом иностранных перевозчиков. От плановых проверок освободят предприятия и организации деятельность которых не относится к категориям чрезвычайно высокого и высокого'],
 ['ПФР приостановило трудовой договор мобилизованных сотрудников для всех кто призван на военную службу по мобилизации Если с мобилизованным работником расторгли договор с указанной даты то нужно издать приказ об отмене увольнения Если растор',
  'ПФР предложил изменить форму СЗВ ТД для граждан, заключивших с февраля по сентября контракт о военной службе или добровольном содействии ВС РФ и уволенных в этот период. Таким образом, граждане заключившие с '],
 ['ИП на УСН с объектом до

In [37]:
inputs_seqs = [el[0] for el in pairs]
target_seqs = [el[1] for el in pairs]

In [38]:
inputs_seqs

['Правительство не будет проводить плановые проверки предприятий и организаций деятельность которых не относится к категориям чрезвычайно высокого и высокого риска и не являются опасными производственными объектами II класса опасности и гидротехническими с',
 'ПФР приостановило трудовой договор мобилизованных сотрудников для всех кто призван на военную службу по мобилизации Если с мобилизованным работником расторгли договор с указанной даты то нужно издать приказ об отмене увольнения Если растор',
 'ИП на УСН с объектом доходы заключил договоры ГПД с физлицами однако работы по ним не выполняли никаких выплат от ИП физическим лицам также не было ФНС рассмотрела такую ситуацию ФНС и',
 'Казначейство временно не размещает на сайте www bus gov ru планы ФХД и отчеты о результатах деятельности и об использовании имущества ведомства Документы будут хранить в закрытом доступе и использовать в т ч при',
 'Организация подала декларацию по НДС в последний день квартала Компания не дождалась оконч

In [39]:
!pip install sentencepiece==0.1.94.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
!pip list

Package                       Version
----------------------------- ----------------------------
absl-py                       1.2.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.10.3
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.8

In [41]:
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration

In [75]:
model = T5ForConditionalGeneration.from_pretrained("google/mt5-base")

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [76]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [77]:
inputs_seqs[0]

'Правительство не будет проводить плановые проверки предприятий и организаций деятельность которых не относится к категориям чрезвычайно высокого и высокого риска и не являются опасными производственными объектами II класса опасности и гидротехническими с'

In [46]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [47]:
import gc
gc.collect()

102

In [88]:
prompt_input_ids, prompt_attention_masks = tokenize(tokenizer, inputs_seqs, 80)
target_input_ids, target_attention_masks = tokenize(tokenizer, target_seqs, 80)

100%|██████████| 397/397 [00:00<00:00, 3622.65it/s]


In [89]:
tokenizer.decode(prompt_input_ids[0])

'Правительство не будет проводить плановые проверки предприятий и организаций деятельность которых не относится к категориям чрезвычайно высокого и высокого риска и не являются опасными производственными объектами II класса опасности и гидротехническими с</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [90]:
prompt_input_ids[0]

tensor([ 21698, 130885,    401,    259,   4466,  12648,   1334,   6824,   8083,
         15435,    657,  74288,    543,    259,    279,  24332,    543,  73607,
           833,   2021,   1827,    401,    259,  25072,   4013,    778,  26242,
          5190,    259, 112122,  62321, 121489,  18600,    893,    259,    279,
         18600,    893,  36301,    308,    259,    279,    401,    259,    396,
         51546,  74061,   5885,  95735,  75081,  17326,   1502,   2052,  10028,
           308,    259, 154268,    259,    279,  89900,  62038,   2566,    388,
             1,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0])

In [ ]:
model.cpu()

In [92]:
tokenizer.decode(
    model.generate(
        input_ids=torch.unsqueeze(prompt_input_ids[0], 0)
    )[0],
    max_new_tokens=128,
    num_beams=100,
    top_p=0.95
)

'<pad> <extra_id_0> и т.д.</s>'

In [93]:
from torch.utils.data import TensorDataset

def get_dataloader(input_ids, attention_masks):
    dataset = TensorDataset(input_ids, attention_masks)
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=2)

    return dataloader

In [94]:
prompt_loader = get_dataloader(prompt_input_ids, prompt_attention_masks)
target_loader = get_dataloader(target_input_ids, target_attention_masks)

In [98]:
device = torch.device("cuda")

In [104]:
model

T5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo

In [108]:
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [107]:
torch.cuda.empty_cache()

In [109]:
model.train()
optimizer = torch.optim.Adam(model.parameters())

for i in range(5):
    for batch in tqdm(zip(prompt_loader, target_loader)):
        prompt_tensors, target_tensors = batch
        prompt_input_ids, prompt_attention_mask = prompt_tensors
        target_input_ids, target_attention_mask = target_tensors

        outputs = model(input_ids=prompt_input_ids.to(device), 
                        attention_mask=prompt_attention_mask.bool().to(device),
                        labels=target_input_ids.to(device), 
                        # output_attentions=target_attention_masks.bool()
                        )
        
        loss = outputs.loss
        loss.backward()
        optimizer.step()

199it [01:16,  2.60it/s]
199it [01:16,  2.61it/s]
199it [01:15,  2.62it/s]
199it [01:15,  2.63it/s]
199it [01:15,  2.62it/s]


In [110]:
prompt_input_ids[0]

tensor([   259, 113591,  97233,   1293,  17556,    324,    259,    279,    259,
           777,  24907,    279,   5332,    587,   6325,   1334,   9296,   3002,
         11542,   1293,    259,    679,   2913,  34832,    688,    259,  24494,
           396,  19383,  17623, 107363,    657,    778,    259, 100149,    259,
         14056,    259,   4690,  10405,   8056,    388,    992,  25162,  14093,
           817,    259,    279,  36886,   7384, 118725,    259, 103760,    324,
         13703,   2013,    315,    259,  80034,   8050,    778,  76963,    259,
             1,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0])

In [114]:
tokenizer.decode(model.generate(input_ids=torch.unsqueeze(prompt_input_ids[0], 0).to(device),  max_new_tokens=128, top_p=0.3)[0])

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> прекразан<pad>из<pad><pad> необходимо<pad> получ<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>,<pad> в<pad><pad><pad><pad><pad> получ<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> необходимошее<pad><pad><pad><pad><pad> за<pad> заРегион<pad><pad><pad>москов<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>счаст<pad>ную<pad><pad><pad><pad><pad><pad><pad><pad>из прев<pad><pad> за<pad><pad><pad>мск<pad><pad><pad>,<pad><pad>ровать<pad> уплближасчита<pad><pad><pad><pad><pad><pad><pad><pad> за<pad>обмен'

In [112]:
tokenizer.decode(prompt_input_ids[0])

'Мобилизованные граждане и их семьи могут получить особые кредитные каникулы С октября действуют поправки к ТК РФ которые связаны с военной службой и мобилизацией Последние изменения в Трудовой кодекс </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'